<h1 align="center">Lecture</h1>

In [39]:
# Большое спасибо Саше Панину, https://github.com/justheuristic

## Рекуррентные Нейронные Сети

<img src="pic/rnn.png" width="600">

## Backprop Through Time

<img src="pic/bptt.png" width="600">


## Truncated Backprop Through Time

<img src="pic/tbptt.png" width="600">


## Char RNN (Embeding Layer vs One Hot Encoding)

<img src="pic/crnn.png" width="600">


## Captioning (Embeding Layer vs One Hot Encoding)

<img src="pic/cap.png" width="600">

<h1 align="center">Char RNN</h1>

In [ ]:
import os
import theano
import lasagne
import numpy as np
import theano.tensor as T

## Прочитаем кучу законов

In [ ]:
#тут будет текст
corpora = ""

for fname in os.listdir("codex"):
    with open("codex/"+fname) as fin:
        corpora += fin.read().decode('cp1251')

In [ ]:
print 'всего букв', len(corpora), 'тип', type(corpora)

In [ ]:
corpora[500:800]

In [ ]:
print corpora[500:800]

## Предобработаем текст

In [ ]:
# тут будут все уникальные токены (буквы, цифры)
tokens = <Все уникальные символы в тексте>
tokens = list(tokens)

In [ ]:
# проверка на количество таких символов. Проверено на Python 2.7.11 Ubuntux64. 
assert len(tokens) == 102

In [ ]:
# словарь символ -> его номер
token_to_id = ...

# словарь номер символа -> сам символ
id_to_token = ...

# Преобразуем всё в токены
corpora_ids = [заменить все символы в corpora на их id]

In [ ]:
def sample_random_batches(source, n_batches=10, seq_len=20):
    X_batch, y_batch = np.zeros((n_batches, seq_len)), np.zeros(n_batches)
    
    for i in xrange(n_batches):
        pos = np.random.randint(0, source.size - seq_len)
        X_batch[i, :] = source[pos:pos+seq_len]
        y_batch[i] = source[pos+seq_len]

    return X_batch, y_batch

In [ ]:
X, y = sample_random_batches(corpora_ids, 4, 5)
print X
print y

## Соберём нейросеть

Вам нужно создать нейросеть, которая принимает на вход последовательность из seq_length токенов, и выдаёт вероятности для seq_len+1-ого.

Общий шаблон архитектуры такой сети:
* Вход
* Обработка входа
* Рекуррентная нейросеть
* Вырезание последнего состояния
* Обычная нейросеть
* Выходной слой, который предсказывает вероятности весов.

Для обработки входных данных можно использовать либо EmbeddingLayer

In [ ]:
seq_length = 5 # Как далеко распространяются градиенты
grad_clip = 10 # Максимальный модуль градиента
input_sequence, target_values = T.matrix('input sequence', 'int32'),  T.ivector('target y')

In [ ]:
net = lasagne.layers.InputLayer(shape=(None, None), input_var=input_sequence)

net = ваша нейронка

net = последний слой, возвращающий веростности для всех len(tokens) вариантов для y 

In [ ]:
# Веса модели
weights = lasagne.layers.get_all_params(net, trainable=True)
print weights

In [ ]:
network_output = Выход нейросети

In [ ]:
loss = Функция потерь - можно использовать простую кроссэнтропию.
updates = Ваш любимый численный метод

## Компилируем всякое-разное

In [ ]:
# обучение
train = theano.function([input_sequence, target_values], loss, updates=updates, allow_input_downcast=True)

# функция потерь без обучения
compute_cost = theano.function([input_sequence, target_values], loss, allow_input_downcast=True)

# Вероятности с выхода сети
probs = theano.function([input_sequence], network_output, allow_input_downcast=True)

## Генерируем свои законы

 * случайно пропорционально вероятности,
 * только слова максимальной вероятностью

In [ ]:
def max_sample_fun(probs):
    return np.argmax(probs) 

def proportional_sample_fun(probs)
    """Сгенерировать следующий токен (int32) по предсказанным вероятностям.
    
    probs - массив вероятностей для каждого токена
    
    Нужно вернуть одно целове число - выбранный токен - пропорционально вероятностям
    """
    
    return номер выбранного слова

In [ ]:
# The next function generates text given a phrase of length at least SEQ_LENGTH.

def generate_sample(sample_fun, seed_phrase=None, N=200):
    if seed_phrase is None:
        start = np.random.randint(0,len(corpora)-seq_length)
        seed_phrase = corpora[start:start+seq_length]
        print "Using random seed:",seed_phrase
    while len(seed_phrase) < seq_length:
        seed_phrase = " "+seed_phrase
    if len(seed_phrase) > seq_length:
        seed_phrase = seed_phrase[len(seed_phrase)-seq_length:]
    assert type(seed_phrase) is unicode
           
    sample_ix = []
    x = map(lambda c: token_to_id.get(c,0), seed_phrase)
    x = np.array([x])

    for i in range(N):
        # Pick the character that got assigned the highest probability
        ix = sample_fun(probs(x).ravel())
        # Alternatively, to sample from the distribution instead:
        # ix = np.random.choice(np.arange(vocab_size), p=probs(x).ravel())
        sample_ix.append(ix)
        x[:,0:seq_length-1] = x[:,1:]
        x[:,seq_length-1] = 0
        x[0,seq_length-1] = ix 

    random_snippet = seed_phrase + ''.join(id_to_token[ix] for ix in sample_ix)    
    print("----\n %s \n----" % random_snippet)

## Обучение модели

В котором вы можете подёргать параметры или вставить свою генерирующую функцию.

In [ ]:
n_epochs=100 #сколько всего эпох
batches_per_epoch = 1000 # раз в сколько эпох печатать примеры 
batch_size=100 #сколько цепочек обрабатывать за 1 вызов функции обучения

print("Training ...")
for epoch in xrange(n_epochs):
    print "Генерируем текст в пропорциональном режиме"
    generate_sample(proportional_sample_fun,None)
    
    print "Генерируем текст в жадном режиме (наиболее вероятные буквы)"
    generate_sample(max_sample_fun,None)

    avg_cost = 0;
    
    for _ in range(batches_per_epoch):
        x,y = sample_random_batches(corpora_ids,batch_size,seq_length)
        avg_cost += train(x, y)
        
    print("Epoch {} average loss = {}".format(epoch, avg_cost / batches_per_epoch))

## Конституция нового мирового правительства

In [ ]:
seed = u"Каждый человек должен"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)

In [ ]:
seed = u"В случае неповиновения"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)

# не забывайте оставлять отзывы 
# о лекции https://goo.gl/gMeYNL о семинаре https://goo.gl/5hlPD0 :)